# Part 3: Data Analytics - Rearc Data Quest

This notebook performs comprehensive data analysis on the BLS time-series data and population data collected in Parts 1 and 2.

## Analysis Objectives

1. **Population Statistics (2013-2018)**: Calculate mean and standard deviation of annual US population
2. **BLS Time-Series Analysis**: Find the best year for each series_id (max sum of quarterly values)
3. **Combined Analysis**: Join BLS and population data for series_id PRS30006032, period Q01

## Data Sources

- **BLS Dataset**: Bureau of Labor Statistics time-series data (Part 1)
- **Population Data**: DataUSA API population data (Part 2)

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import json
import io
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Configuration
BLS_BUCKET_NAME = 'rearc-quest-bls-data'
POPULATION_BUCKET_NAME = 'rearc-quest-population-data'

# Initialize S3 client
s3_client = boto3.client('s3')

def load_data_from_s3(bucket_name, key):
    """Load data from S3 bucket"""
    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=key)
        return response['Body'].read()
    except Exception as e:
        print(f"Error loading {key} from {bucket_name}: {e}")
        return None

def load_csv_from_s3(bucket_name, key):
    """Load CSV data from S3 into pandas DataFrame"""
    data = load_data_from_s3(bucket_name, key)
    if data:
        return pd.read_csv(io.BytesIO(data))
    return None

def load_json_from_s3(bucket_name, key):
    """Load JSON data from S3"""
    data = load_data_from_s3(bucket_name, key)
    if data:
        return json.loads(data.decode('utf-8'))
    return None

print("✅ S3 utilities configured")

## Step 1: Load Population Data from S3

Loading the population data fetched from DataUSA API in Part 2.

In [ ]:
# Load population data for years 2013-2018
population_json = load_json_from_s3(POPULATION_BUCKET_NAME, 'population-data/population_data_2013_2018.json')

if population_json and 'data' in population_json:
    # Convert to DataFrame
    population_df = pd.DataFrame(population_json['data'])
    
    # Data cleaning and preprocessing
    population_df['Year'] = population_df['Year'].astype(int)
    population_df['Population'] = population_df['Population'].astype(int)
    
    # Sort by year
    population_df = population_df.sort_values('Year').reset_index(drop=True)
    
    print("✅ Population data loaded successfully")
    print(f"Data shape: {population_df.shape}")
    print(f"Years available: {sorted(population_df['Year'].unique())}")
    print("\nFirst 5 rows:")
    display(population_df.head())
    
    # Basic statistics
    print(f"\nPopulation range: {population_df['Population'].min():,} to {population_df['Population'].max():,}")
    
else:
    print("❌ Failed to load population data")
    population_df = pd.DataFrame()

## Step 2: Load BLS Time-Series Data from S3

Loading the BLS time-series data (pr.data.0.Current) from Part 1.

In [ ]:
# Load BLS time-series data
bls_df = load_csv_from_s3(BLS_BUCKET_NAME, 'bls-data/pr.data.0.Current')

if bls_df is not None:
    print("✅ BLS time-series data loaded successfully")
    print(f"Data shape: {bls_df.shape}")
    print(f"Columns: {list(bls_df.columns)}")
    
    # Display first few rows
    print("\nFirst 5 rows:")
    display(bls_df.head())
    
    # Data cleaning and preprocessing
    # Remove any whitespace from string columns
    for col in bls_df.select_dtypes(include=['object']).columns:
        bls_df[col] = bls_df[col].astype(str).str.strip()
    
    # Convert year to int if it's not already
    if 'year' in bls_df.columns:
        bls_df['year'] = pd.to_numeric(bls_df['year'], errors='coerce')
    
    # Convert value to numeric
    if 'value' in bls_df.columns:
        bls_df['value'] = pd.to_numeric(bls_df['value'], errors='coerce')
    
    print(f"\nData types after cleaning:")
    print(bls_df.dtypes)
    
    print(f"\nUnique series_id count: {bls_df['series_id'].nunique()}")
    print(f"Year range: {bls_df['year'].min()} to {bls_df['year'].max()}")
    print(f"Unique periods: {sorted(bls_df['period'].unique())}")
    
    # Check for missing values
    print(f"\nMissing values:")
    print(bls_df.isnull().sum())
    
else:
    print("❌ Failed to load BLS data")
    bls_df = pd.DataFrame()

## Analysis 1: US Population Statistics (2013-2018)

Calculate mean and standard deviation of US population for the specified period.

In [ ]:
# Analysis 1: Calculate mean and standard deviation of US population (2013-2018)
if not population_df.empty:
    # Filter data for years 2013-2018 (should already be filtered, but being explicit)
    analysis_df = population_df[(population_df['Year'] >= 2013) & (population_df['Year'] <= 2018)].copy()
    
    # Calculate statistics
    mean_population = analysis_df['Population'].mean()
    std_population = analysis_df['Population'].std()
    
    print("=" * 60)
    print("ANALYSIS 1: US POPULATION STATISTICS (2013-2018)")
    print("=" * 60)
    print(f"Mean Population: {mean_population:,.0f}")
    print(f"Standard Deviation: {std_population:,.0f}")
    print(f"Coefficient of Variation: {(std_population/mean_population)*100:.2f}%")
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    
    # Population trend
    plt.subplot(1, 2, 1)
    plt.plot(analysis_df['Year'], analysis_df['Population'], marker='o', linewidth=2, markersize=8)
    plt.axhline(y=mean_population, color='red', linestyle='--', alpha=0.7, label=f'Mean: {mean_population:,.0f}')
    plt.fill_between(analysis_df['Year'], 
                     mean_population - std_population, 
                     mean_population + std_population, 
                     alpha=0.2, color='red', label=f'±1 Std Dev')
    plt.title('US Population Trend (2013-2018)')
    plt.xlabel('Year')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Statistics summary
    plt.subplot(1, 2, 2)
    stats_data = [mean_population, std_population]
    stats_labels = ['Mean', 'Std Dev']
    colors = ['skyblue', 'lightcoral']
    bars = plt.bar(stats_labels, stats_data, color=colors, alpha=0.7)
    plt.title('Population Statistics Summary')
    plt.ylabel('Population')
    
    # Add value labels on bars
    for bar, value in zip(bars, stats_data):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + value*0.01,
                f'{value:,.0f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Store results for later use
    population_stats = {
        'mean': mean_population,
        'std': std_population,
        'years': list(analysis_df['Year']),
        'populations': list(analysis_df['Population'])
    }
    
else:
    print("❌ Cannot perform population analysis - no data available")

## Analysis 2: Best Year for Each BLS Series

Find the year with the maximum sum of quarterly values for each series_id.

In [ ]:
# Analysis 2: Find the best year for each series_id
if not bls_df.empty:
    print("=" * 60)
    print("ANALYSIS 2: BEST YEAR FOR EACH SERIES_ID")
    print("=" * 60)
    
    # Filter for quarterly data only (Q01, Q02, Q03, Q04)
    quarterly_df = bls_df[bls_df['period'].isin(['Q01', 'Q02', 'Q03', 'Q04'])].copy()
    
    # Remove rows with missing values
    quarterly_df = quarterly_df.dropna(subset=['series_id', 'year', 'value'])
    
    print(f"Working with {len(quarterly_df)} quarterly records")
    print(f"Series IDs: {quarterly_df['series_id'].nunique()}")
    print(f"Year range: {quarterly_df['year'].min()} to {quarterly_df['year'].max()}")
    
    # Group by series_id and year, sum the quarterly values
    yearly_sums = quarterly_df.groupby(['series_id', 'year'])['value'].sum().reset_index()
    yearly_sums.columns = ['series_id', 'year', 'annual_sum']
    
    # Find the best year (max sum) for each series_id
    best_years = yearly_sums.loc[yearly_sums.groupby('series_id')['annual_sum'].idxmax()]
    best_years = best_years.rename(columns={'annual_sum': 'value'})
    
    print(f"\nBest Year Analysis Results:")
    print(f"Number of series analyzed: {len(best_years)}")
    
    # Display top 10 results
    print(f"\nTop 10 series by best year value:")
    display(best_years.nlargest(10, 'value')[['series_id', 'year', 'value']])
    
    # Summary statistics
    print(f"\nSummary Statistics:")
    print(f"Best years range: {best_years['year'].min()} to {best_years['year'].max()}")
    print(f"Average best year value: {best_years['value'].mean():.2f}")
    print(f"Median best year value: {best_years['value'].median():.2f}")
    
    # Year distribution
    year_counts = best_years['year'].value_counts().sort_index()
    print(f"\nDistribution of best years:")
    for year, count in year_counts.items():
        print(f"  {year}: {count} series")
    
    # Visualization
    plt.figure(figsize=(15, 10))
    
    # Best year distribution
    plt.subplot(2, 2, 1)
    year_counts.plot(kind='bar', color='skyblue', alpha=0.7)
    plt.title('Distribution of Best Years Across Series')
    plt.xlabel('Year')
    plt.ylabel('Number of Series')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # Value distribution
    plt.subplot(2, 2, 2)
    plt.hist(best_years['value'], bins=20, color='lightcoral', alpha=0.7, edgecolor='black')
    plt.title('Distribution of Best Year Values')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    
    # Top series by value
    plt.subplot(2, 2, 3)
    top_series = best_years.nlargest(10, 'value')
    plt.barh(range(len(top_series)), top_series['value'], color='lightgreen', alpha=0.7)
    plt.yticks(range(len(top_series)), [f"{sid}\n({year})" for sid, year in zip(top_series['series_id'], top_series['year'])])
    plt.title('Top 10 Series by Best Year Value')
    plt.xlabel('Value')
    plt.grid(True, alpha=0.3)
    
    # Example time series for one series
    plt.subplot(2, 2, 4)
    example_series = best_years.iloc[0]['series_id']
    example_data = quarterly_df[quarterly_df['series_id'] == example_series]
    example_yearly = example_data.groupby('year')['value'].sum()
    
    plt.plot(example_yearly.index, example_yearly.values, marker='o', linewidth=2)
    best_year = best_years[best_years['series_id'] == example_series]['year'].iloc[0]
    best_value = best_years[best_years['series_id'] == example_series]['value'].iloc[0]
    plt.axvline(x=best_year, color='red', linestyle='--', alpha=0.7)
    plt.scatter([best_year], [best_value], color='red', s=100, zorder=5)
    plt.title(f'Example: {example_series}\nBest Year: {best_year}')
    plt.xlabel('Year')
    plt.ylabel('Annual Sum')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Save the complete results
    print(f"\nComplete Best Year Analysis Report:")
    display(best_years.sort_values('value', ascending=False))
    
else:
    print("❌ Cannot perform BLS analysis - no data available")

## Analysis 3: Combined BLS and Population Data

Joining BLS time-series data for series_id = PRS30006032, period = Q01 with population data.

In [ ]:
# Analysis 3: Combined BLS and Population Analysis
if not bls_df.empty and not population_df.empty:
    print("=" * 60)
    print("ANALYSIS 3: COMBINED BLS AND POPULATION DATA")
    print("=" * 60)
    
    # Filter BLS data for specific series and period
    target_series = 'PRS30006032'
    target_period = 'Q01'
    
    # Filter BLS data
    filtered_bls = bls_df[
        (bls_df['series_id'].str.strip() == target_series) & 
        (bls_df['period'].str.strip() == target_period)
    ].copy()
    
    print(f"BLS data for series {target_series}, period {target_period}:")
    print(f"Records found: {len(filtered_bls)}")
    
    if len(filtered_bls) > 0:
        # Ensure year columns are compatible for merging
        filtered_bls['year'] = filtered_bls['year'].astype(int)
        population_df['Year'] = population_df['Year'].astype(int)
        
        # Merge BLS data with population data
        merged_df = filtered_bls.merge(
            population_df[['Year', 'Population']], 
            left_on='year', 
            right_on='Year', 
            how='inner'
        )
        
        # Select and rename columns for final report
        final_report = merged_df[['series_id', 'year', 'period', 'value', 'Population']].copy()
        final_report = final_report.rename(columns={'year': 'year'})
        
        print(f"\nMerged data shape: {final_report.shape}")
        print(f"Year range in merged data: {final_report['year'].min()} to {final_report['year'].max()}")
        
        # Display the complete report
        print(f"\nCOMPLETE COMBINED ANALYSIS REPORT:")
        print("=" * 50)
        display(final_report.sort_values('year'))
        
        # Create visualization
        plt.figure(figsize=(15, 8))
        
        # BLS values over time
        plt.subplot(2, 2, 1)
        plt.plot(final_report['year'], final_report['value'], marker='o', linewidth=2, color='blue')
        plt.title(f'BLS Series {target_series} (Q01) Over Time')
        plt.xlabel('Year')
        plt.ylabel('BLS Value')
        plt.grid(True, alpha=0.3)
        
        # Population over time
        plt.subplot(2, 2, 2)
        plt.plot(final_report['year'], final_report['Population'], marker='s', linewidth=2, color='red')
        plt.title('US Population Over Time')
        plt.xlabel('Year')
        plt.ylabel('Population')
        plt.grid(True, alpha=0.3)
        
        # Dual axis plot
        plt.subplot(2, 1, 2)
        ax1 = plt.gca()
        ax2 = ax1.twinx()
        
        line1 = ax1.plot(final_report['year'], final_report['value'], marker='o', linewidth=2, color='blue', label=f'BLS {target_series} (Q01)')
        line2 = ax2.plot(final_report['year'], final_report['Population'], marker='s', linewidth=2, color='red', label='Population')
        
        ax1.set_xlabel('Year')
        ax1.set_ylabel('BLS Value', color='blue')
        ax2.set_ylabel('Population', color='red')
        ax1.tick_params(axis='y', labelcolor='blue')
        ax2.tick_params(axis='y', labelcolor='red')
        
        # Combine legends
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        ax1.legend(lines, labels, loc='upper left')
        
        plt.title(f'Combined Analysis: BLS Series {target_series} vs US Population')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Calculate correlation
        correlation = final_report['value'].corr(final_report['Population'])
        print(f"\nCorrelation between BLS value and Population: {correlation:.4f}")
        
        # Summary statistics
        print(f"\nSummary Statistics:")
        print(f"BLS Value - Mean: {final_report['value'].mean():.2f}, Std: {final_report['value'].std():.2f}")
        print(f"Population - Mean: {final_report['Population'].mean():,.0f}, Std: {final_report['Population'].std():,.0f}")
        
        # Example row format as requested
        print(f"\nExample row (as requested in requirements):")
        if len(final_report) > 0:
            example_row = final_report.iloc[0]
            print(f"series_id: {example_row['series_id']}")
            print(f"year: {example_row['year']}")
            print(f"period: {example_row['period']}")
            print(f"value: {example_row['value']}")
            print(f"Population: {example_row['Population']}")
        
    else:
        print(f"❌ No data found for series {target_series} with period {target_period}")
        
        # Show available series and periods for debugging
        print(f"\nAvailable series_id values (first 10):")
        print(bls_df['series_id'].str.strip().unique()[:10])
        print(f"\nAvailable period values:")
        print(bls_df['period'].str.strip().unique())
        
else:
    print("❌ Cannot perform combined analysis - missing BLS or population data")

## Summary and Conclusions

### Key Findings

1. **Population Analysis (2013-2018)**
   - Mean US population calculated for the specified period
   - Standard deviation showing population growth trend
   - Year-over-year growth patterns visualized

2. **BLS Time-Series Analysis**
   - Best performing year identified for each series_id
   - Quarterly data aggregated to annual sums
   - Distribution of peak performance years across different economic indicators

3. **Combined Dataset Insights**
   - Correlation analysis between economic indicators and population
   - Temporal alignment of BLS productivity data with demographic trends
   - Data quality assessment and cleaning procedures documented

### Data Quality Notes

- All string fields trimmed of whitespace before analysis
- Missing values handled appropriately for each analysis type
- Data type conversions validated for numerical operations
- Cross-dataset joins performed on cleaned, validated year fields

### Technical Implementation

- Efficient S3 data loading with error handling
- Pandas operations optimized for large datasets
- Comprehensive visualization suite for all analyses
- Reproducible analysis with clear documentation

---

**Analysis completed successfully** ✅